In [13]:
import pandas as pd

def calculate_bid_scores_pandas(full_score, bids_df):
    """
    使用Pandas计算评标价（适合大量数据）
    
    Args:
        full_score (float): 价格得分满分
        bids_df (pd.DataFrame): 包含'company'和'price'列的DataFrame
    
    Returns:
        pd.DataFrame: 包含得分结果的DataFrame
    """
    # 复制数据避免修改原数据
    df = bids_df.copy()
    
    # 按价格排序
    df = df.sort_values('price')
    
    # 计算基准价
    if len(df) >= 5:
        # 去掉最高价和最低价
        trimmed_df = df.iloc[1:-1]
        benchmark = trimmed_df['price'].mean() * 0.98
    else:
        benchmark = df['price'].mean() * 0.98
    
    # 计算得分
    def calculate_score(row):
        price = row['price']
        if price > benchmark:
            return full_score - (price - benchmark)/benchmark * 100 * 0.2
        elif price < benchmark:
            return full_score - (benchmark - price)/benchmark * 100 * 0.1
        else:
            return full_score
    
    df['score'] = df.apply(calculate_score, axis=1)
    df['score'] = df['score'].apply(lambda x: max(0, x))  # 确保得分不为负
    df['difference'] = df['price'] - benchmark
    df['is_higher'] = df['price'] > benchmark
    
    # 添加基准价信息
    df['benchmark'] = benchmark
    
    return df

# 使用示例
def pandas_example():
    """Pandas版本使用示例"""
    # 创建示例数据
    data = {
        'company': ['中建三局基础设施建设投资有限公司', '北京市市政四建设工程有限责任公司', 
                    '武汉天创建设集团有限公司', '中国十九冶集团有限公司', '武汉市市政建设集团有限公司', 
                    '中铁四院集团工程运维有限责任公司'],
        'price': [199610004.33,195428656.48, 
                  198749647, 199673509.54, 162554543.8, 
                  195750289.27]
    }
    df = pd.DataFrame(data)
    
    full_score = 50
    
    # 计算得分
    result_df = calculate_bid_scores_pandas(full_score, df)
    
    print("评标价计算结果 (Pandas版本):")
    print("=" * 50)
    print(f"基准价: {result_df['benchmark'].iloc[0]:.2f}")
    print("\n各公司得分:")
    print(result_df[['company', 'price', 'difference', 'score']])
    
    return result_df

# 运行Pandas示例
pandas_example()

评标价计算结果 (Pandas版本):
基准价: 193436956.28

各公司得分:
            company         price    difference      score
4     武汉市市政建设集团有限公司  1.625545e+08 -3.088241e+07  48.403490
1  北京市市政四建设工程有限责任公司  1.954287e+08  1.991700e+06  49.794072
5  中铁四院集团工程运维有限责任公司  1.957503e+08  2.313333e+06  49.760818
2      武汉天创建设集团有限公司  1.987496e+08  5.312691e+06  49.450706
0  中建三局基础设施建设投资有限公司  1.996100e+08  6.173048e+06  49.361751
3       中国十九冶集团有限公司  1.996735e+08  6.236553e+06  49.355185


,company,price,score,difference,is_higher,benchmark
4,武汉市市政建设集团有限公司,1.625545e+08,48.403490,-3.088241e+07,False,1.934370e+08
1,北京市市政四建设工程有限责任公司,1.954287e+08,49.794072,1.991700e+06,True,1.934370e+08
5,中铁四院集团工程运维有限责任公司,1.957503e+08,49.760818,2.313333e+06,True,1.934370e+08
2,武汉天创建设集团有限公司,1.987496e+08,49.450706,5.312691e+06,True,1.934370e+08
0,中建三局基础设施建设投资有限公司,1.996100e+08,49.361751,6.173048e+06,True,1.934370e+08
3,中国十九冶集团有限公司,1.996735e+08,49.355185,6.236553e+06,True,1.934370e+08
